In [1]:
import pandas as pd
import datetime
import requests

In [2]:
start = datetime.datetime.strptime("01-08-2018", "%d-%m-%Y")
end = datetime.datetime.strptime("01-09-2018", "%d-%m-%Y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

array = []

for date in date_generated:
    #print (date.strftime("%Y-%m-%d"))
    URL = 'https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=18713d003c6f45d6870185944210109&q=New%20York&format=json&date=%27+date.strftime(%22%Y-%m-%d%22)+%27&tp=24'
    wetter = requests.get('https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=18713d003c6f45d6870185944210109&q=New York&format=json&date='+date.strftime("%Y-%m-%d")+'&tp=1')
    custom = {"date":wetter.json()["data"]["weather"][0]["date"] } 
    custom.update({"hourly":wetter.json()["data"]["weather"][0]["hourly"]})
    #custom.update({"sunHour" : wetter.json()["data"]["weather"][0]["sunHour"]})
    df = pd.json_normalize(custom,record_path=["hourly"],meta=["date"])
    df = pd.DataFrame(df,columns=["date","time","tempC","precipMM","humidity","weatherDesc","windspeedKmph"])
    array.append(df)

In [3]:
new_weather = pd.concat(array,axis=0,ignore_index = True)

In [5]:
new_weather['time'] = new_weather['time'] / 100

In [4]:
new_weather['time'] = new_weather['time'].astype(int)

In [6]:

def getvalue(value):    
    return  value[0]["value"]

def getValue(data):
    #print(data)
    return getvalue(data['weatherDesc'])

In [7]:
new_weather['weatherDesc2'] = new_weather.apply(getValue,axis=1)

In [8]:
new_weather['weatherDesc2'].drop_duplicates().to_numpy().tolist()

['Overcast',
 'Light rain shower',
 'Moderate or heavy rain shower',
 'Clear',
 'Partly cloudy',
 'Patchy rain possible',
 'Sunny',
 'Torrential rain shower',
 'Patchy light drizzle',
 'Light drizzle',
 'Cloudy',
 'Moderate rain',
 'Moderate rain at times']

In [9]:
new_weather = new_weather.drop("weatherDesc",axis=1)

In [10]:
new_weather["date"] =  pd.to_datetime(new_weather['date'], errors='coerce')

In [11]:
new_weather['year'] = new_weather['date'].dt.year
new_weather['month'] = new_weather['date'].dt.month
new_weather['day'] = new_weather['date'].dt.day

In [12]:
new_weather.drop("date",axis=1).to_csv('weatherforWestChambers082018.csv',index=False)

In [54]:
for i in ["tempC","sunHour","precipMM","FeelsLikeC","humidity"]:
    new_weather[i] = new_weather[i].astype(float)

In [55]:
new_weather2 = new_weather.groupby([new_weather['date'].dt.year,new_weather['date'].dt.isocalendar().week]).agg('mean')

In [58]:
new_weather2.head()

tempC   sunHour  precipMM  FeelsLikeC   humidity
date week                                                     
2015 1     6.000000  6.525000  6.450000   -1.250000  71.000000
     2    -3.571429  7.214286  1.071429  -13.714286  69.428571
     3     0.571429  7.214286  8.200000   -6.714286  70.571429
     4     2.714286  8.142857  3.685714   -3.714286  72.714286
     5    -1.571429  7.628571  3.828571  -11.285714  84.714286

In [57]:
new_weather2.to_csv('weather.csv')

In [5]:
import datetime
date = datetime.date(2015,1,1)
wetter = requests.get('https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=18713d003c6f45d6870185944210109&q=New York&format=json&date='+date.strftime("%Y-%m-%d")+'&tp=1')

In [6]:
wetter.json()["data"]["weather"][0]["hourly"]

[{'time': '0',
  'tempC': '-3',
  'tempF': '26',
  'windspeedMiles': '10',
  'windspeedKmph': '15',
  'winddirDegree': '247',
  'winddir16Point': 'WSW',
  'weatherCode': '113',
  'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0008_clear_sky_night.png'}],
  'weatherDesc': [{'value': 'Clear'}],
  'precipMM': '0.0',
  'precipInches': '0.0',
  'humidity': '52',
  'visibility': '10',
  'visibilityMiles': '6',
  'pressure': '1022',
  'pressureInches': '30',
  'cloudcover': '6',
  'HeatIndexC': '-3',
  'HeatIndexF': '26',
  'DewPointC': '-12',
  'DewPointF': '11',
  'WindChillC': '-10',
  'WindChillF': '15',
  'WindGustMiles': '14',
  'WindGustKmph': '23',
  'FeelsLikeC': '-10',
  'FeelsLikeF': '15',
  'uvIndex': '1'},
 {'time': '100',
  'tempC': '-4',
  'tempF': '26',
  'windspeedMiles': '11',
  'windspeedKmph': '18',
  'winddirDegree': '253',
  'winddir16Point': 'WSW',
  'weatherCode': '113',
  'weatherIconUrl': [{'value': 'http://cdn.worldw